<a href="https://colab.research.google.com/github/rehmanaly0051/CGAN-Model-Training/blob/main/Conditional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
#from google.colab import drive
#drive.mount('/content/drive')

## 1. Dataset Preprocessing

In [40]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

### Defining Dataset class

In [41]:
class CubiCasa5kDataset(Dataset):
    def __init__(self, data_list, root_dir, transforms=None):
        # Ensure data_list is not None and not empty
        self.data_list = data_list if data_list is not None else []
        self.root_dir = root_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx): # Correcting the typo in method name
        img_path = os.path.join(self.root_dir, self.data_list[idx] + '_original.png')
        label_path = os.path.join(self.root_dir, self.data_list[idx] + '_scaled.png')

        image = Image.open(img_path).convert('RGB')
        label = Image.open(label_path).convert('RGB')

        if self.transforms:
            image = self.transforms(image)
            label = self.transforms(label)

        return image, label


### Dataset path

In [42]:
# dataset path
dataset_path = '/content/drive/MyDrive/cubicasa5k'

### Loading Train, Test and Val labels

In [43]:
def load_labels(file_path):
    # Ensure file exists and is not empty before processing
    if not os.path.exists(file_path):
        print(f"Warning: File not found: {file_path}")
        return [] # Return empty list if file not found
    with open(file_path, 'r') as f:
        lines = f.readlines()
        labels = [line.strip() for line in lines]
        if not labels: # Check for empty labels list
            print(f"Warning: File is empty: {file_path}")
    return labels

train_data = load_labels(os.path.join(dataset_path, 'train.txt'))
test_data = load_labels(os.path.join(dataset_path, 'test.txt'))
val_data = load_labels(os.path.join(dataset_path, 'val.txt'))

### Defining transfomations

In [44]:
transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5 ,),(0.5 ,))
])

### Creating dataset

In [45]:
dataset_train = CubiCasa5kDataset(train_data, dataset_path, transforms)
dataset_test = CubiCasa5kDataset(test_data, dataset_path, transforms)
dataset_val = CubiCasa5kDataset(val_data, dataset_path, transforms)

### Creating dataloaders

In [46]:
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False)

### Displaying dataset size

In [47]:
print(f'Train Samples :{len(dataset_train)}',
      f'Test Samples : {len(dataset_test)}',
      f'Val Samples : {len(dataset_val)}')

Train Samples :4200 Test Samples : 400 Val Samples : 400


## CGAN floorplan Generation

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torch.utils.data import DataLoader

### Defining Generator class

In [51]:
class Generator(nn.Module):
  def __init__(self, input_dim, output_channels):
    super(Generator, self).__init__()
    self.model = nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Linear(512, 1024),
        nn.ReLU(),
        nn.Linear(1024, output_channels),
        nn.Tanh()
    )

  def forward(self, z):
    return self.model(z)

### Defining Discriminator class

In [53]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img)

### Setting parameters

In [54]:
latent_dim = 100
image_dim = 256 * 256 * 3
batch_size = 16
num_epochs = 100
learning_rate = 0.0002

### Initializing Models

In [55]:
generator = Generator(latent_dim, image_dim)
discriminator = Discriminator(image_dim)

gan_loss = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_d = optim.Adam(discriminator.parameters(), lr=learning_rate)

### Training Loop

In [ ]:
epoches = 50
device = torch.device('cuda' if torch.cuda.is_avaibale else 'cpu')
generator.to(device)
discriminator.to(device)

dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

for epoch in range(epoches):
  for i, (real_imgs, _) in enumerate(dataloader):
    real_img = real_imgs.view(batch_size, -1).to(device)

    # Train discriminator
    optimizer_d.zero_grad()
    z = torch.randn(batch_size, latent_dim).to(device)
    fake_imgs = generator(z).detach()

    real_loss = gan_loss(discriminator(real_imgs), torch.ones(batch_size, 1).to(device))
    fake_loss = gan_loss(discriminator(fake_imgs), torch.zeros(batch_size, 1).to(device))
    d_loss = real_loss + fake_loss
    d_loss.backward()
    optimizer_d.step()

    # Train generator
    optimizer_g.zero_grad()
    fake_imgs = generator(z)
    g_loss = gan_loss(discriminator(fake_imgs), torch.ones(batch_size, 1).to(device))
    g_loss.backward()
    optimizer_g.step()

  print(f'Epoch {epoch}/{epoches} | D Loss : {d_loss.item()} G Loss : {g_loss.item()}')

  # creating output folder
  output_folder = '/content/drive/MyDrive/output_images'
  os.makedirs(output_folder, exist_ok=True)

  if epoch % 10 == 0:
    save_path = os.path.join(output_folder, f'generated_epoch_{epoch}.png')
    vutils.save_image(fake_imgs.view(batch_size, 3, 256, 256), save_path, normalize=True)
